In [1]:
import torch

import argparse
import dill
import glob
import importlib
import os
from tqdm import tqdm

from criterion import Criterion
from data import *
from dataset import *
from test import initilize_testing_loader, evaluate
from utils import parse_config, set_seeds, set_devices, load_pretrained_weight, load_checkpoint

In [2]:
parser = argparse.ArgumentParser(description='Visualizing')
parser.add_argument('--trial', default='shapenet_with_norm.rscnn.0001', type=str,
                    help='The trial name with its version, e.g., modelnet40.dgcnn.0001.')
parser.add_argument('--show_configs', default=False, type=bool,
                    help='Whether to print the config at the program beginning.')
parser.add_argument('--show_loss_details', default=False, type=bool,
                    help='Whether to show all loss details.')
parser.add_argument('--separator_bar', default='*' * 100, type=str,
                    help='Separator bar.')
parser.add_argument('--max_float_digits', default=4, type=int,
                    help='Limit the max digits to display.')
parser.add_argument('--pretrained_weight_path', default='', type=str,
                    help='The pre-trained weight path.')
parser.add_argument('--checkpoint_path', default='best', type=str,
                    help='Checkpoint path. "best" means the best checkpoint of saved ones.')

_StoreAction(option_strings=['--checkpoint_path'], dest='checkpoint_path', nargs=None, const=None, default='best', type=<class 'str'>, choices=None, help='Checkpoint path. "best" means the best checkpoint of saved ones.', metavar=None)

In [ ]:
args = parser.parse_args()
parse_config(args)

    
# Dynamically import the model class, Net.
model_path = os.path.join(args.trial_dir, 'model')
model_package = importlib.import_module(model_path.replace('/', '.'))

# Set the environment.
set_seeds(args.seed + args.breakpoint_continuation * random.randint(1, 9e3))
device = set_devices(args.cuda_devices, args.separator_bar)

# Load the dataset.
loader = initilize_testing_loader(args)

# Set the model.
model = model_package.Net(loader.dataset.num_classes(args.task_type),
                            **args.net_arguments).to(device)
load_pretrained_weight(args.pretrained_weight_path, model)
load_checkpoint(args.trial_dir, args.checkpoint_path, model)

criterion = Criterion(args.task_type,
                          args.criterion,
                          args.show_loss_details,
                          args.max_float_digits).to(device)

testing_results = evaluate(args, loader, model, device, criterion)
all_sample_detils = criterion.all_sample_details
criterion.reset()

In [ ]:
all_sample_detils